In [0]:
#download the data and save it to book.txt
!wget https://www.gutenberg.org/files/1661/1661-0.txt -O book.txt

#read the file in text string
text = open('book.txt', 'r', encoding='utf-8').read()
text = text.lower()

#create list of sentences
sentences = text.split('\n')

#import dependencies to preprocess the text data and making sequences
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#initialize the tokenizer, which can work char by char
tokenizer = Tokenizer(oov_token='<UNK>')

tokenizer.fit_on_texts(sentences)

vocab_size = len(tokenizer.word_index) + 1
vocab_size

input_sequences = []
for sequence in sequences:
  for i in range(1, len(sequence)):
    n_gram_sequence = sequence[:i+1]
    input_sequences.append(n_gram_sequence)

print(input_sequences[0], input_sequences[1], input_sequences[2], input_sequences[3])

#find the maximum length among sequences
max_seq_len = max([len(seq) for seq in input_sequences])
max_seq_len

#we will keep the last value of the sequence as our target label and all values before that as input to sequence model

#pad the sequences to ensure that they are all of same length
padded_sequences = pad_sequences(input_sequences, maxlen = max_seq_len)

print(padded_sequences[0], padded_sequences[1])

import numpy as np
padded_sequences = np.array(padded_sequences)

print(len(padded_sequences[0]))
print(len(padded_sequences[1]))

#prepare training sequences and labels
x = padded_sequences[:, : -1]
labels = padded_sequences[:, -1]

labels.shape

#to one hot encode the labels
y = tf.keras.utils.to_categorical(labels, num_classes=vocab_size)

x.shape

#x = x.reshape(x.shape[0], x.shape[1], 1)

y.shape

#import dependencies for defining the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

#define and compile the model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_seq_len-1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
model.summary()

#initialize the callback for early stopping the training if there is not at least 1% improvement in the accuracy 
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'acc', min_delta=0.01)

model.fit(x, y, epochs=50, verbose=1, batch_size=512, callbacks=[es])

#Time to become storyteller!
seed_text = "I could not help laughing at the ease with which he explained his process of deduction"          
next_words = 100
  
for _ in range(next_words):
  sequence = tokenizer.texts_to_sequences([seed_text])
  padded = pad_sequences(sequence, maxlen=max_seq_len-1)
  predicted = model.predict_classes(padded, verbose=0)
  output_word = ''
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += ' ' + output_word
print(seed_text)

#let's look at how loss and accuracy changed while training
import matplotlib.pyplot as plt
history = model.history
acc = history.history['acc']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

--2020-03-15 07:42:05--  https://www.gutenberg.org/files/1661/1661-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607788 (594K) [text/plain]
Saving to: ‘book.txt’

book.txt            100%[===================>] 593.54K   498KB/s    in 1.2s    

2020-03-15 07:42:07 (498 KB/s) - ‘book.txt’ saved [607788/607788]

